In [5]:
from notion_client import Client
from pprint import pprint

notion_token = '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'
notion_page_id = '1dd7d0050794806fafbcf4fdb5fa0c0f'

def main():
    client = Client(auth=notion_token)

    page_response = client.pages.retrieve(notion_page_id)

    pprint(page_response, indent=2)

if __name__ == '__main__':
    main()
          

APIResponseError: Could not find page with ID: 1dd7d005-0794-806f-afbc-f4fdb5fa0c0f. Make sure the relevant pages and databases are shared with your integration.

In [8]:
from notion_client import Client
from pprint import pprint
import re

notion_token = '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'

def format_page_id(page_id):
    """Format page ID to proper UUID format with hyphens"""
    # Remove any existing hyphens and spaces
    clean_id = re.sub(r'[-\s]', '', page_id)
    
    # Add hyphens in UUID format: 8-4-4-4-12
    if len(clean_id) == 32:
        formatted_id = f"{clean_id[:8]}-{clean_id[8:12]}-{clean_id[12:16]}-{clean_id[16:20]}-{clean_id[20:]}"
        return formatted_id
    else:
        return page_id

def extract_page_id_from_url(url):
    """Extract page ID from Notion URL"""
    # Pattern to match Notion URLs and extract the ID
    pattern = r'([a-f0-9]{32}|[a-f0-9]{8}-[a-f0-9]{4}-[a-f0-9]{4}-[a-f0-9]{4}-[a-f0-9]{12})(?:\?|$)'
    match = re.search(pattern, url, re.IGNORECASE)
    
    if match:
        return format_page_id(match.group(1))
    return None

def main():
    client = Client(auth=notion_token)
    
    # Try different page ID formats
    page_ids_to_try = [
        '1dd7d0050794806fafbcf4fdb5fa0c0f',  # Original
        format_page_id('1dd7d0050794806fafbcf4fdb5fa0c0f'),  # Formatted
        # Add your actual page URL here if you have it
        # extract_page_id_from_url('your-full-notion-url-here')
    ]
    
    for page_id in page_ids_to_try:
        if page_id is None:
            continue
            
        print(f"Trying page ID: {page_id}")
        
        try:
            page_response = client.pages.retrieve(page_id)
            print("✅ Success! Page retrieved:")
            pprint(page_response, indent=2)
            return  # Exit if successful
            
        except Exception as e:
            print(f"❌ Failed with page ID {page_id}: {str(e)}")
            print()
    
    print("📋 Troubleshooting steps:")
    print("1. Make sure your integration has access to the page:")
    print("   - Open the page in Notion")
    print("   - Click 'Share' → 'Invite' → Add your integration")
    print("2. Get the correct page ID from your page URL")
    print("3. Verify your integration token is correct")

if __name__ == '__main__':
    main()

Trying page ID: 1dd7d0050794806fafbcf4fdb5fa0c0f
❌ Failed with page ID 1dd7d0050794806fafbcf4fdb5fa0c0f: Could not find page with ID: 1dd7d005-0794-806f-afbc-f4fdb5fa0c0f. Make sure the relevant pages and databases are shared with your integration.

Trying page ID: 1dd7d005-0794-806f-afbc-f4fdb5fa0c0f
❌ Failed with page ID 1dd7d005-0794-806f-afbc-f4fdb5fa0c0f: Could not find page with ID: 1dd7d005-0794-806f-afbc-f4fdb5fa0c0f. Make sure the relevant pages and databases are shared with your integration.

📋 Troubleshooting steps:
1. Make sure your integration has access to the page:
   - Open the page in Notion
   - Click 'Share' → 'Invite' → Add your integration
2. Get the correct page ID from your page URL
3. Verify your integration token is correct


In [11]:
from notion_client import Client
from pprint import pprint
import re

notion_token = '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'

def test_integration_access():
    """Test if the integration token is valid and can access anything"""
    client = Client(auth=notion_token)
    
    print("🔍 Testing integration access...")
    print("-" * 50)
    
    try:
        # Try to search for pages (this should work if integration has any access)
        response = client.search(query="", page_size=10)
        
        if response.get('results'):
            print("✅ Integration is working! Found accessible pages:")
            for i, page in enumerate(response['results'][:5], 1):
                title = "Untitled"
                if page.get('properties') and page['properties'].get('title'):
                    title_prop = page['properties']['title']
                    if title_prop.get('title') and len(title_prop['title']) > 0:
                        title = title_prop['title'][0]['plain_text']
                
                print(f"   {i}. {title} (ID: {page['id']})")
            
            print(f"\n📊 Total accessible pages: {len(response['results'])}")
            return response['results']
        else:
            print("⚠️  Integration is valid but has no access to any pages")
            print("   You need to share pages with your integration first")
            return []
            
    except Exception as e:
        print(f"❌ Integration test failed: {str(e)}")
        print("\n🔧 Possible issues:")
        print("   1. Invalid API token")
        print("   2. Integration doesn't exist")
        print("   3. Token expired")
        return None

def format_page_id(page_id):
    """Clean and format page ID"""
    # Remove URL parameters and special characters
    clean_id = re.sub(r'[?#].*$', '', str(page_id))
    clean_id = re.sub(r'[-\s]', '', clean_id)
    
    if len(clean_id) == 32:
        return f"{clean_id[:8]}-{clean_id[8:12]}-{clean_id[12:16]}-{clean_id[16:20]}-{clean_id[20:]}"
    return clean_id

def try_page_access(page_id):
    """Try to access a specific page"""
    client = Client(auth=notion_token)
    formatted_id = format_page_id(page_id)
    
    print(f"\n🎯 Trying to access page: {formatted_id}")
    
    try:
        page = client.pages.retrieve(formatted_id)
        print("✅ Success! Page details:")
        
        # Extract title
        title = "Untitled"
        if page.get('properties') and page['properties'].get('title'):
            title_prop = page['properties']['title']
            if title_prop.get('title') and len(title_prop['title']) > 0:
                title = title_prop['title'][0]['plain_text']
        
        print(f"   Title: {title}")
        print(f"   Created: {page.get('created_time', 'Unknown')}")
        print(f"   Last edited: {page.get('last_edited_time', 'Unknown')}")
        
        return page
        
    except Exception as e:
        print(f"❌ Failed: {str(e)}")
        return None

def main():
    print("🚀 Notion Integration Debugger")
    print("=" * 50)
    
    # Test 1: Check if integration works at all
    accessible_pages = test_integration_access()
    
    # Test 2: Try the specific page IDs
    test_page_ids = [
        '1dd7d0050794806fafbcf4fdb5fa0c0f',
        '1dd9a3a6132180b6bccfc7cfa39bd39b'
    ]
    
    print("\n" + "=" * 50)
    print("🎯 Testing specific page access...")
    
    for page_id in test_page_ids:
        try_page_access(page_id)
    
    # Instructions
    print("\n" + "=" * 50)
    print("📋 Next Steps:")
    print("1. If no pages are accessible, share pages with your integration:")
    print("   • Open page in Notion → Share → Invite → [Your Integration Name]")
    print("2. Check your integration at: https://www.notion.so/my-integrations")
    print("3. Make sure you're using the correct page ID from the URL")
    print("4. The page must be shared with your integration to be accessible")

if __name__ == '__main__':
    main()

🚀 Notion Integration Debugger
🔍 Testing integration access...
--------------------------------------------------
✅ Integration is working! Found accessible pages:
   1. Feature Engineering Concepts (ID: 1dd9a3a6-1321-8087-954b-e0787b2299ad)
   2. Linear Regression (ID: 1e79a3a6-1321-802a-ae5a-f77cbafb9e0d)
   3. Evaluation Metrics for Linear Regression (ID: 1e79a3a6-1321-80fd-b59b-e5fc249bb7df)
   4. Multiple Linear Regression (ID: 1e79a3a6-1321-8018-ba1a-cb5b49e68844)
   5. Simple Linear Regression (ID: 1e79a3a6-1321-80cc-a1d0-e0031a14976d)

📊 Total accessible pages: 5

🎯 Testing specific page access...

🎯 Trying to access page: 1dd7d005-0794-806f-afbc-f4fdb5fa0c0f
❌ Failed: Could not find page with ID: 1dd7d005-0794-806f-afbc-f4fdb5fa0c0f. Make sure the relevant pages and databases are shared with your integration.

🎯 Trying to access page: 1dd9a3a6-1321-80b6-bccf-c7cfa39bd39b
❌ Failed: Could not find page with ID: 1dd9a3a6-1321-80b6-bccf-c7cfa39bd39b. Make sure the relevant pages an

In [1]:
from notion_client import Client
import json
import re
import os 
from dotenv import load_dotenv

load_dotenv(override=True)
notion_token = os.getenv('NOTION_TOKEN')

In [6]:
client = Client(auth=notion_token)
res = client.search(
    query="",
    page_size=100,
    filter={
        'property': 'object', 
        'value': 'page'
    }
)

In [11]:
type(res)

dict

In [14]:
for i,j in res.items():
    print("**********")
    print(i)
    print(j)
    print('\n')

**********
object
list


**********
results
[{'object': 'page', 'id': '1f59a3a6-1321-803d-8c2d-cee3d827b70b', 'created_time': '2025-05-16T04:11:00.000Z', 'last_edited_time': '2025-05-27T05:50:00.000Z', 'created_by': {'object': 'user', 'id': 'f4a5e46a-8ad1-4907-aef8-b70d61151ca2'}, 'last_edited_by': {'object': 'user', 'id': 'f4a5e46a-8ad1-4907-aef8-b70d61151ca2'}, 'cover': None, 'icon': {'type': 'external', 'external': {'url': 'https://www.notion.so/icons/layers_blue.svg'}}, 'parent': {'type': 'page_id', 'page_id': '1f59a3a6-1321-80bf-b882-f64fe45aed6d'}, 'archived': False, 'in_trash': False, 'properties': {'title': {'id': 'title', 'type': 'title', 'title': [{'type': 'text', 'text': {'content': 'Redis UseCases In LLM Application', 'link': None}, 'annotations': {'bold': False, 'italic': False, 'strikethrough': False, 'underline': False, 'code': False, 'color': 'default'}, 'plain_text': 'Redis UseCases In LLM Application', 'href': None}]}}, 'url': 'https://www.notion.so/Redis-UseCases-In-

In [17]:
pages = []
for result in res.get('results', []):
    if result.get('object')=='page':
        extract_title(result)

{'object': 'page', 'id': '1f59a3a6-1321-803d-8c2d-cee3d827b70b', 'created_time': '2025-05-16T04:11:00.000Z', 'last_edited_time': '2025-05-27T05:50:00.000Z', 'created_by': {'object': 'user', 'id': 'f4a5e46a-8ad1-4907-aef8-b70d61151ca2'}, 'last_edited_by': {'object': 'user', 'id': 'f4a5e46a-8ad1-4907-aef8-b70d61151ca2'}, 'cover': None, 'icon': {'type': 'external', 'external': {'url': 'https://www.notion.so/icons/layers_blue.svg'}}, 'parent': {'type': 'page_id', 'page_id': '1f59a3a6-1321-80bf-b882-f64fe45aed6d'}, 'archived': False, 'in_trash': False, 'properties': {'title': {'id': 'title', 'type': 'title', 'title': [{'type': 'text', 'text': {'content': 'Redis UseCases In LLM Application', 'link': None}, 'annotations': {'bold': False, 'italic': False, 'strikethrough': False, 'underline': False, 'code': False, 'color': 'default'}, 'plain_text': 'Redis UseCases In LLM Application', 'href': None}]}}, 'url': 'https://www.notion.so/Redis-UseCases-In-LLM-Application-1f59a3a61321803d8c2dcee3d827b